In [1]:
import tensorflow.keras as keras
(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()

In [2]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
import numpy as np

xtrain = xtrain.astype('float32') / 255
xtest = xtest.astype('float32') / 255

xtrain = np.reshape(xtrain, (xtrain.shape[0], 28, 28, 1))
xtest = np.reshape(xtest, (xtest.shape[0], 28, 28, 1))

print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)


In [4]:
import sqlite3

conn = sqlite3.connect('fashion_mnist.db')

conn.execute('''CREATE TABLE IF NOT EXISTS images
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             image BLOB NOT NULL,
             label INTEGER NOT NULL);''')

for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtrain[i]), ytrain[i]])

conn.commit()

for i in range(xtest.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtest[i]), ytest[i]])

conn.commit()

In [5]:
import sqlite3
conn = sqlite3.connect('fashion_mnist.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

import pandas as pd
data=pd.read_sql_query('SELECT * FROM images',conn )

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

# Veri kümesini yükleme
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Görüntüleri ve etiketleri normalize etme
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Modeli oluşturma
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Modeli derleme
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Modeli eğitme
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Modeli değerlendirme
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


Epoch 1/5
938/938 [==============================] - 8s 9ms/step - loss: 0.5450 - accuracy: 0.8020
Epoch 2/5
938/938 [==============================] - 8s 9ms/step - loss: 0.3366 - accuracy: 0.8770
Epoch 3/5
938/938 [==============================] - 9s 9ms/step - loss: 0.2898 - accuracy: 0.8949
Epoch 4/5
938/938 [==============================] - 9s 9ms/step - loss: 0.2598 - accuracy: 0.9058
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.2659 - accuracy: 0.9072
Test accuracy: 0.9071999788284302
